<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">TP : La cuisine du système</h1>

On souhaite observer le fonctionnement de l'ordonnanceur du systéme d'exploitation en prenant la métaphore d'une cuisine de restaurant.

On considère des programmes Python qui seront des recettes de cuisine.  
Nous utiliserons les fichiers d'un répertoire `ingredients` pour gérer notre garde-manger, des fichiers d'un repertoire `ustensiles` pour gérer les ressources de la cuisine (four, batteur, etc) et enfin les fichiers d'un répertoire `plats` pour compter les plats produits par les cuisiniers.  
Nous nous interesserons aux accès concurrents qui peuvet être faits sur ces ressources, aux problèmes qui peuvent en découler et à l'utilisation des verrous pour éviter ces problèmes.  
Enfin, avec les verrous, nous illustrerons une situaation d'interblocage.

## Ordonnancement et concurrence
Dans un premier temps, nous allons réaliser une bibliothèque [`actions.py`](Fichiers/actions.py) qui implémente une fonction pour chacun des gestes de cuisine que les recettes peuvent demander de réaliser.

Par exemple, le fichier [`actions.py`](Fichiers/actions.py) définit la fonction :

In [ ]:
def prendre_un_abricot(pourquoi):
    print("Prendre un abricot,", pourquoi)
    return prendre("ingredients/abricot")

Cette fonction affiche un message indiquant qu'un abricot est utilisé et en précise l'usage, puis appelle `prendre()` qui décrémente le nombre d'abricots écrits dans le fichier `abricots` du repertoire `ingredients`.

La fonction `prendre()` ouvre le fichier passé en paramètre, lit le nombre stocké dans ce fichier, décrémente ce nombre, puis écrit cette valeur du nombre d'ingrédients disponibles dans le même fichier :

In [ ]:
def prendre(filename):
    file = open(filename)
    quantite = int(file.readline())
    file.close()
    if quantite > 0: 
       quantite = quantite - 1
    file = open(filename,"w")
    file.write(str(quantite))
    file.close()
    return quantite

Nous allons réaliser une première recette qui consiste simplement, pour le cuisinier, à manger un abricot.

Cette recette sera implémentée dans un fichier [`manger_un_abricot.py`](Fichiers/manger_un_abricot.py) comme suit :

In [ ]:
import actions

abricots = actions.prendre_un_abricot("pour le manger !")
print("Il reste", abricots)

Pour que cette recette puisse être réalisée, il faut, préalablement, créer le répertoire `ingredients`, puis mettre des abricots dans ce *garde-manger*.

    utilisateur@machine:~$ mkdir ingredients

    utilisateur@machine:~$ echo 100 > ingredients/abricot

L'éxecution de la recette produit la trace suivante : 

    utilisateur@machine:~$ python3 manger_un_abricot.py
    Prendre un abricot, pour le manger !
    Il reste 99


Il est possible de faire faire cette recette à deux cuisiniers, simultanément, avec la (double) commande suivante :

    utilisateur@machine:~$ python3 manger_un_abricot.py & python3 manger_un_abricot.py & 

Si tout se passe bien, cela roduit la trace suivante : 

    [1] 2617469
    [2] 2617470

    utilisateur@machine:~$ Prendre un abricot, pour le manger !
    Il reste 98
    Prendre un abricot, pour le manger !
    Il reste 97

    [1]-  Fini                    python3 manger_un_abricot.py
    [2]+  Fini                    python3 manger_un_abricot.py

Cette trace montre bien que deux programmes se sont exécutés en même temps.  
Chacun a décrémenté le compteur du garde-manger et il ne reste que 97 abricots.

Pourtant, en re-exécutant plusieurs fois cette double commande, il est possible que l'on obtienne une trace *anormale* telle que celle-ci :

    utilisateur@machine:~$ python3 manger_un_abricot.py & python3 manger_un_abricot.py & 
    [1] 2710223
    [2] 2710224
    utilisateur@machine:~$ Prendre un abricot, pour le manger !
    Prendre un abricot, pour le manger !
    Il reste 96
    Il reste 96

    [1]-  Fini                    python3 manger_un_abricot.py
    [2]+  Fini                    python3 manger_un_abricot.py

    utilisateur@machine:~$ Prendre un abricot, pour le manger !
    Prendre un abricot, pour le manger !
    Il reste 96
    Il reste 96

    [1]-  Fini                    python3 manger_un_abricot.py
    [2]+  Fini                    python3 manger_un_abricot.py

Il semble alors que le programme ne fonctionne pas correctement.  
En effet, deux exécutions se sont déroulées et elles ont toutes deux décrémentées le nombre d'abricots, et pourtant, un seul abricot a été *mangé*.

Cette situation n'est pas systématique : elle se produit parfois, et, parfois, elle ne se produit pas...  
On dit que ce comportement n'est pas *déterministe* (ou qu'il est *indéterministe*).

1 . Pour comprendre ce qui s'est produit, considérer la fonction `prendre()`, qui a été utilisée par les deux programmes.  
Proposer un ordonnancement des deux programmes qui aurait pu faire passer le nombre d'abricots de 97 à 95, puis un autre qui aurait pu faire passer le nombre d'abricots de 97 à 96. 

2 . Sachant que les deux situations sont possibles, expliquer pourquoi le comportement observé n'est pas déterministe.

Il apparaît donc que le fait de lancer deux programmes en même temps compromet leur bon fonctionnement.  
Pour éviter ce problème, il pourrait sembler légitime de s'interdire de lancer deux programmes en même temps.

3 . En filant la métaphore de la cuisine, expliquer pourquoi le restaurant n'a pas intérêt à réaliser les recettes les unes après les autres s'il y a deux cuisiniers, et même s'il n'y en a qu'un.

Cet exemple montre que la fonction `prendre()`